# Read and extract H, S, U, Cv @ 25, 305 and 705 K

In [ ]:
import os
import warnings
import json
import pandas as pd
from pymatgen.core import Structure

warnings.filterwarnings("ignore")

**Please download the required data to get the complete dataset from [here](https://springernature.figshare.com/articles/dataset/Parsed_phonon_data/5649298?backTo=%2Fcollections%2FHigh-throughput_Density-Functional_Perturbation_Theory_phonons_for_inorganic_materials%2F3938023&file=9850858)**

Then provide the path to these json files in the `phonon_data_path` variable below. For demonstration we have provided just one example entry in this repository

In [ ]:
phonon_data_path = "example_phonon_db_files/phonon_data/"

In [ ]:
parent_dir = os.getcwd()

In [ ]:
os.chdir(phonon_data_path)

In [ ]:
df_data = {}
for f in os.listdir():
    with open(f) as file:
        data = json.load(file)
        columns = data["flags"]
        struct = Structure.from_str(data["metadata"]["structure"], fmt="cif")
        factor = struct.composition.get_reduced_composition_and_factor()[0].num_atoms
        try:
            for temp in (25, 305, 705):
                index = data["thermo"]["temperature"].index(temp)
                u = data["thermo"]["internal"][index] * (6.241506363094E+21 / (6.02214076e+23*factor)) # meV/atom
                s = data["thermo"]["entropy"][index] * (6.241506363094E+21 / (6.02214076e+23*factor)) # meV/K/atom
                h = data["thermo"]["helmholtz_energy"][index] * (6.241506363094E+21 / (6.02214076e+23*factor)) # meV/atom
                cv = data["thermo"]["C_v"][index] * (6.241506363094E+21 / (6.02214076e+23*factor))   # meV/K/atom
    
                columns.update({f"U_{temp}": u, f"S_{temp}": s, f"H_{temp}": h, f"Cv_{temp}": cv})
        except KeyError:
            pass

            
        df_data[f.strip(".json")] = columns

In [ ]:
df = pd.DataFrame.from_dict(df_data, orient="index")

In [ ]:
stable_entries = df[df.has_neg_fr==False] # filter for stable entries
stable_entries.drop(index=['mp-6270'], inplace=True) # exclude entry for which lobster calculation failed

In [ ]:
targets = ["Cv_25", "Cv_305", "Cv_705",
           "H_25", "H_305", "H_705", 
           "S_25", "S_305", "S_705", 
           "U_25", "U_305", "U_705"]

In [ ]:
os.chdir(parent_dir)

In [ ]:
# Save target jsons
for target in targets:
    stable_entries.loc[:, [target]].to_json(f"{target}.json")